# Mini Group Project
Credit: Nguyen Dinh Hung and Nguyen Thi My Le \
DO NOT COPY WITHOUT PERMISSION

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data cleaning

In [ ]:
# import data
raw_df = pd.read_csv(os.path.join(dirname, "CarPrice_Assignment.csv"))
raw_df.sample(10)

In [ ]:
raw_df.dtypes

### Drop car_ID

In [ ]:
raw_df.drop(['car_ID'], axis=1, inplace=True)

### Remove Duplicate values in car names
- nissan == Nissan
- toyota == toyouta
- vokswagen == volkswagen == vw
- mazda == maxda
- porcshce == porsche


In [ ]:
raw_df.CarName = raw_df.CarName.str.split(' ', expand=True)

In [ ]:
raw_df['CarName'] = raw_df['CarName'].replace(
    {'maxda': 'mazda', 
     'nissan': 'Nissan', 
     'porcshce': 'porsche', 
     'toyouta': 'toyota', 
    'vokswagen': 'volkswagen', 'vw': 'volkswagen'
    }
)

In [ ]:
# raw_df.describe()
set(raw_df['CarName'])
# raw_df

In [ ]:
raw_df['symboling']=raw_df['symboling'].astype('str')

# Exploratory Data Analysis

In [ ]:
num_cols = raw_df.describe().columns
raw_df.describe()

### Car names

In [ ]:
# carName_df = pd.DataFrame(raw_df['CarName'])
raw_df['CarName'].value_counts().plot(kind='bar')

According to the dataset:

Toyota is the most popular car \
Mercury is the least popular car

### Fuel type
Most car uses Gas over diesel

In [ ]:
df = pd.DataFrame(raw_df['fueltype'].value_counts())
plot = df.plot.pie(y='fueltype', figsize=(5, 5))
plot

### Price distribution of Cars
Price max: 45400 \
Price min: 5118
- Right skewed
- maximum number of cars are in range of 20000

In [ ]:
sns.distplot(raw_df['price'])

### Price distribution of Fuel type (gas vs. diesel)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(221)
plt.hist(raw_df[raw_df['fueltype']== 'gas']['price'], edgecolor='black', bins=5)
plt.title('Price distribution of gas vehicles (5 bins)')
plt.xlabel('price')
plt.ylabel('frequency')

plt.subplot(222)
plt.hist(raw_df[raw_df['fueltype']== 'diesel']['price'], edgecolor='black', bins=5)
plt.title('Price distribution of diesel vehicles (5 bins)')
plt.xlabel('price')
plt.ylabel('frequency')

plt.subplot(223)
plt.hist(raw_df[raw_df['fueltype']== 'gas']['price'], edgecolor='black', bins=10)
plt.xlabel('price')
plt.ylabel('frequency')
plt.title('Price distribution of gas vehicles (10 bins)')

plt.subplot(224)
plt.hist(raw_df[raw_df['fueltype']== 'diesel']['price'], edgecolor='black', bins=10)
plt.title('Price distribution of diesel vehicles (10 bins)')
plt.xlabel('price')
plt.ylabel('frequency')

plt.tight_layout()
plt.show()

## Missing Value

In [ ]:
print(len(raw_df)) # 205

In [ ]:
clean_df = raw_df[num_cols].dropna()
print(len(clean_df)) # 205 -> Data already cleaned

## Hệ số tương quan: (Correlation Coeficient)

In [ ]:
correlation = raw_df.corr()
correlation

# heat map
import seaborn as sns

mask = np.zeros_like(correlation, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


fig = plt.figure(figsize=(12, 8))
sns.heatmap(correlation, mask=mask, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

1 : Tương Quan Đồng Biến \
0 : Độc lập, không tương quan \
-1: Tương Quan Nghịch Biến 

### Categorical using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

# creating instance of labelencoder
labelencoder = LabelEncoder()
raw_df['CarName_Cat'] = labelencoder.fit_transform(raw_df['CarName'])
raw_df['fueltype_Cat'] = labelencoder.fit_transform(raw_df['fueltype'])
raw_df

In [ ]:
raw_df[['CarName', 'CarName_Cat']].groupby(by='CarName').mean()


Get only the features I have analyzed

In [ ]:
cols = list(num_cols)
cols.append('CarName_Cat')
cols.append('fueltype_Cat')
cols

clean_df = raw_df[cols].copy()

clean_df

# Build Model
Because we haven't studied how to Encode Categorical data, we'll just be using numberical data for now (and Car name + fueltype)


## Split data

In [ ]:
from sklearn.model_selection import train_test_split
# print(cols.remove('price'))
print(cols)
X = clean_df[cols]
y = clean_df[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

## Multiple Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
sns.pairplot(clean_df, x_vars=cols[0:5], y_vars='price', size=7, aspect=0.7, kind='reg')

In [ ]:
sns.pairplot(clean_df, x_vars=cols[5:10], y_vars='price', size=7, aspect=0.7, kind='reg')

In [ ]:
sns.pairplot(clean_df, x_vars=cols[10:], y_vars='price', size=7, aspect=0.7, kind='reg')

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error

y_pred = linreg.predict(X_train)
print("R2-score: %.2f" % r2_score(y , y_pred))
# calculate MAE, MSE, RMSE
print(mean_absolute_error(y, y_pred))
print(mean_squared_error(y, y_pred))
print(np.sqrt(mean_squared_error(y, y_pred)))

## Evaluation

# Conclusion